# Yanling single-cell analysis

In [ ]:
from cellassign import assign_cats
import gzip
import itertools as itl
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import os
import pandas as pd
import scanpy as sc
import scanpy.external as sce
import seaborn as sns
import triku as tk

In [ ]:
# Palettes for UMAP gene expression

magma = [plt.get_cmap('magma')(i) for i in np.linspace(0,1, 80)]
magma[0] = (0.88, 0.88, 0.88, 1)
magma = mpl.colors.LinearSegmentedColormap.from_list("", magma[:65])

In [ ]:
mpl.rcParams['figure.dpi'] = 200

In [ ]:
def plot_batch_abundance(adatax, plot_status=False):
    samples, clusters, list_prop = sorted(dict.fromkeys(adatax.obs['batch'])), sorted(dict.fromkeys(adatax.obs['leiden'])), []
    product = itl.product(*[samples, clusters])

    correction_factor = {sample: len(adatax)/(len(samples) * np.sum(adatax.obs['batch'] == sample)) for sample in samples}

    for sample, cluster in product:
        list_prop.append(correction_factor[sample] * 
                         len(adatax[(adatax.obs['leiden'] == cluster) & (adatax.obs['batch'] == sample)])/
                         len(adatax[adatax.obs['leiden'] == cluster]))

    df = pd.DataFrame({'x':clusters * len(samples), 'y':list_prop, 'hue':sorted(samples * len(clusters))})
    if plot_status:
        df['hue_status'] = [i[0] for i in df['hue']]
        fig, axs = plt.subplots(1, 2, figsize=(16,4))
        sns.barplot(x='x', y='y', hue='hue', data=df, ax=axs[0])
        sns.barplot(x='x', y='y', hue='hue_status', data=df, ax=axs[1])
    else:
        fig, axs = plt.subplots(1, 1, figsize=(8,4))
        sns.barplot(x='x', y='y', hue='hue', data=df, ax=axs)

In [ ]:
fastq_dir = f'{os.getcwd()}/fastq' 
mouse_gencode_dir = "/media/seth/SETH_DATA/SETH_Alex/Programs/mouse_GRCm38_gencode.v31"

In [ ]:
prefixes = ['KOD11', 'KOD12', 'WT1', 'WT2']

## Dataset processing [**DO NOT RUN IF NOT NECESSARY**]

The reads are produced based on the 10X v3 library preparation kit, which consists of a cell barcode of 16 bp, a UMI of 12 bp, and a read of 91 bp.
To process the files we are going to first trim the reads to that length, and then preprocess them using `loompy fastq` to get the files with the read information.

In [ ]:
df = pd.DataFrame({'name': prefixes, 'technology': ['10xv3'] * len(prefixes), 'targetnumcells': [1000] * len(prefixes)})
df.to_csv(fastq_dir + '/metadata.tab', sep='\t', index=None)

In [ ]:
for filename_root in prefixes: 
    fileinR1 = gzip.open(f'{fastq_dir}/{filename_root}/long_{filename_root}_L001_R1_001.fastq.gz', 'rt') 
    fileinR2 = gzip.open(f'{fastq_dir}/{filename_root}/long_{filename_root}_L001_R2_001.fastq.gz', 'rt') 
    
    fileoutR1 = open(f'{fastq_dir}/{filename_root}/{filename_root}_L001_R1_001.fastq', 'w') 
    fileoutR2 = open(f'{fastq_dir}/{filename_root}/{filename_root}_L001_R2_001.fastq', 'w') 


    count = 0

    while True: 
        count += 1

        # Get next line from file 
        lineR1 = fileinR1.readline() 
        lineR2 = fileinR2.readline() 

        if count % 4 in [1, 3]:
            fileoutR1.write(lineR1.replace('\n', '') + '\n')
            fileoutR2.write(lineR2.replace('\n', '') + '\n')
        elif count == 2:
            fileoutR1.write(lineR1.replace('\n', '')[:28] + '\n')
            fileoutR2.write(lineR2.replace('\n', '')[:91] + '\n')
        else:
            fileoutR1.write(lineR1.replace('\n', '')[:28] + '\n')
            fileoutR2.write(lineR2.replace('\n', '')[:91] + '\n')


        # if line is empty 
        # end of file is reached 
        if not lineR1: 
            break

    fileinR1.close() 
    fileinR2.close()
    fileoutR1.close()
    fileoutR2.close()
    
    os.system(f"cd {fastq_dir}/{filename_root} && gzip {filename_root}_L001_R1_001.fastq")
    os.system(f"cd {fastq_dir}/{filename_root} && gzip {filename_root}_L001_R2_001.fastq")
    os.system(f"cd {fastq_dir} && loompy fromfq {filename_root}.loom {filename_root} {mouse_gencode_dir} metadata.tab {fastq_dir}/{filename_root}/{filename_root}_L001_R1_001.fastq.gz {fastq_dir}/{filename_root}/{filename_root}_L001_R2_001.fastq.gz")

## Load adatas

In [ ]:
seed = 0

In [ ]:
adata_KOD11 = sc.read(f"{fastq_dir}/KOD11.loom")
adata_KOD11.var_names_make_unique()

adata_KOD12 = sc.read(f"{fastq_dir}/KOD12.loom")
adata_KOD12.var_names_make_unique()

adata_WT1 = sc.read(f"{fastq_dir}/WT1.loom")
adata_WT1.var_names_make_unique()

adata_WT2 = sc.read(f"{fastq_dir}/WT2.loom")
adata_WT2.var_names_make_unique()

In [ ]:
adata_all = sc.AnnData.concatenate(adata_KOD11, adata_KOD12, adata_WT1, adata_WT2, batch_categories=['KO1', 'KO2', 'WT1', 'WT2'])
adata_all.obs['condition'] = [i[:2] for i in adata_all.obs['batch']]

In [ ]:
adata_all.obs['status'] = [i[0] for i in adata_all.obs['batch']]

In [ ]:
for prefix in prefixes:
    adata_all.obs[f'is_{prefix}'] = (adata_all.obs['batch'] == prefix).astype(str)
    adata_all.uns[f'is_{prefix}_colors'] = ['#bcbcbc', '#bc0000']

## Pre-Setting the AnnData files for cellxgene

In [ ]:
adata_all.raw = adata_all

### Basic QC filtering

In this step we are going to look for mitochondrial read and gene expression distribution to see if there are imbalances in the datasets, if they are correctable, and to apply the corrections in that case.

In [ ]:
# Basic QC filtering
adata_all.var['mt'] = adata_all.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata_all, qc_vars=['mt'], percent_top=None, inplace=True)

In [ ]:
sc.pl.violin(adata_all, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)

sc.pl.scatter(adata_all, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(adata_all, x='total_counts', y='n_genes_by_counts')

We see that the read distribution is irregular in the whole dataset, so we are going to plot the same distributions samplewise.

In [ ]:
# PERCENTAGE OF MITOCHONDRIAL READS
fig, ax = plt.subplots(1, 1, figsize=(10, 4))
df = pd.DataFrame({'x': adata_all.obs['batch'], 'y': np.log1p(adata_all.obs['pct_counts_mt'])})
sns.violinplot(x='x', y='y', data=df, ax=ax)

In [ ]:
# NATURAL LOG OF NUMBER OF GENES WITH AT LEAST 1 COUNT
fig, ax = plt.subplots(1, 1, figsize=(10, 4))
df = pd.DataFrame({'x': adata_all.obs['batch'], 'y': adata_all.obs['log1p_n_genes_by_counts']})
sns.violinplot(x='x', y='y', data=df, ax=ax)

In [ ]:
# Apply filters
adata_all = adata_all[
(((adata_all.obs.batch == 'KO1') & (adata_all.obs.log1p_n_genes_by_counts < 8.0) & (adata_all.obs.log1p_n_genes_by_counts > 5.8)) |
 ((adata_all.obs.batch == 'KO2') & (adata_all.obs.log1p_n_genes_by_counts < 8.0) & (adata_all.obs.log1p_n_genes_by_counts > 6.5)) |
 ((adata_all.obs.batch == 'WT1')   & (adata_all.obs.log1p_n_genes_by_counts < 8.0) & (adata_all.obs.log1p_n_genes_by_counts > 6.5)) |
 ((adata_all.obs.batch == 'WT2')   & (adata_all.obs.log1p_n_genes_by_counts < 8.0) & (adata_all.obs.log1p_n_genes_by_counts > 6.5)))]
adata_all = adata_all[adata_all.obs.pct_counts_mt < 10, :]

In [ ]:
sc.pp.filter_genes(adata_all, min_counts=1)
sc.pp.normalize_total(adata_all)
sc.pp.log1p(adata_all)

## Cell type identification

In this step we are going to generate a set of marker genes for different populations, that should be expressed in as many samples as possible. To do that we applied different clusterings to the samples, got the DEGs, and manually determined the set of genes.

Afterwards, to assign the populations with the genes, we run the clustering to obtain many clusters, many of which are merged under the same population.

Then, to check that each population is assigned correctly, we check that the marker genes are expressed in that populations. There might be some cases where two different populations appear as one (there are not enough cells to discern them, or they are mixed for whichever reason), or that a population is assigned despite not expressing all the markers.

In [ ]:
# 5 good genes per cell type
# For immune cell types I used http://rstats.immgen.org/Skyline/skyline.html

dict_celltypes = {'Keratinocyte $Krt5^+$': ['Krt5', 'Fgfr2', 'Lamb3', 'Col7a1', 'Gm17851'], 
                  'Keratinocyte $Krt10^+$': ['Krt10', 'Krt1', 'Them5', 'Endou', 'Klk8'],  # Spninous
                  'Keratinocyte $Lor^+$': ['Lor', 'Nccrp1', 'Trex2', 'Lce1a1', 'Lce1b'],  # Granular
                  'Keratinocyte $Tbx1^+$': ['Dusp6', 'Slc7a8', 'Tbx1', 'Shisa2', 'Ucp2'],
                  'Keratinocyte $Krt28^+$': ['Krt28','Dlx3', 'Krt25', 'Krt27', 'Krt71'],
                  'Keratinocyte $Krt75^+$': ['Gjb2', 'Wnt11', 'Krt75', 'Fzd5', 'Fads3'],
                  'Keratinocyte $Defb6^+$': ['Krt79', 'Defb6', 'Atp6v1c2', 'Nebl', 'Teddm3'],
                  'Keratinocyte $Anln^+$': ['Anln', 'Prc1', 'Cdk1', 'Cenpf', 'Dnph1'],
                  'Keratinocyte $Cidea^+$': ['Cidea', 'Ldhb', 'Aadac', 'Bex1', 'Pparg'],
                  # 'Fibroblast': ['Lum', 'Pdgfra', 'Mfap2', 'Mfap5', 'Clec3b'],
                  'Fibroblast $Cxcl12^+$': ['Cxcl12', 'Htra3', 'C1s1', 'Lol', 'Cygb'],
                  'Fibroblast $Thbs4^+$': ['Thbs4', 'Spon2', 'Fmod', 'Ptgis', 'Cilp2'],
                  'Fibroblast $Cxcl1^+$': ['Cxcl1', 'Tnfaip6', 'Ccl7', 'Has1', 'Cxcl10'],
                  'Fibroblast $Clec3b^+$': ['Clec3b', 'Fbn1', 'Pi16', 'Scara5', 'Ugp2'],
                  'Fibroblast $Col8a1^+$': ['Col8a1', 'Eid1', 'Arap1', 'Gpr153', 'Igfbp2'],
                  'Fibroblast $Coch^+$': ['Coch', 'Crabp1', 'Fbn2', 'Emid1', 'Wfdc1'],                  
                  'Fibroblast $Rab37^+$': ['Rab37', 'Col22a1', 'F13a1', 'Htra4', 'Tspan15'],
                  'Fibroblast $Chf^+$': ['Cfh', 'Alpl', 'Lifr', 'Sp7', 'Spp1'],
                  'Fibroblast $Ptgs2^+$': ['Il1rl1', 'Ptgs2', 'Nr4a2', 'Gxylt2', 'Lum'],
                  'Fibroblast $Serpine2^+$': ['Serpine2', 'Shox2', 'Wif1', 'Gm48159', 'Col23a1'],
                  'Chondrogenic fibroblast': ['Col9a1', 'Col9a2', 'Scrg1', 'Hapln1', 'Trpv4'],
                  'Vascular endothelial cell': ['Pecam1', 'Cldn5', 'Cdh5', 'Ptprb', 'Tie1'],
                  'Lymphatic endothelial cell': ['Mmrn1', 'Ccl21a', 'Prox1', 'Lyve1', 'Flt4'],
                  'Perivascular cell $Inpp4b^+$': ['Rgs5', 'Myh11', 'Aoc3', 'Inpp4b', 'Mrvi1'],
                  'Perivascular cell $Il6^+$': ['Rgs5', 'Myh11', 'Il6', 'Procr', 'Ngf'],
                  'Schwann cell': ['Prx', 'Mbp', 'Mpz', 'Ncmap', 'Cldn19'], 
                  'Glial cell': ['Gfra3', 'Plp1', 'Scn7a', 'Cdh19', 'Adam23'],  
                  'Melanocyte': ['Pmel', 'Mlana', 'Dct'],
                  'Skeletal muscle': ['Msc', 'Myod1', 'Cdh15', 'Peg3', 'Dag1'], 
                  'Red blood cell': ['Hba-a1', 'Hbb-bt', 'Hbb-bs', 'Car2', 'Rhd'],
                  'T cell': ['Cd3d', 'Cd3e', 'Ifngr1', 'Klf2', 'Cd27'],
                  'T cell (ILC/gd)?': ['Cd7', 'Cd3e', 'Ctsw', 'Cd3d', 'Cd3g'],
                  'B cell': ['Rrm2', 'Rpa3', 'Cd79b', 'Dntt', 'Cd79a'],
                  'Plasma cell': ['Ighm', 'Igkc', 'Cd79b', 'Iglc1', 'Iglc2'],
                  'NK cell': ['Cd3d', 'Cd3e', 'Nkg7', 'Klrk1', 'Trdv4'],
                  'Macrophage': ['C1qa', 'C1qc', 'Wfdc17', 'Pf4', 'Folr2'],
                  'Monocyte': ['Wfdc17', 'Csf1r', 'F10', 'Ly6c2', 'Gsr'],
                  'Neutrophil': ['S100a8', 'S100a9', 'Camp', 'Ltf', 'Chil3'],
                  'Neutrophil*': ['S100a9', 'Acod1', 'Il1f9', 'Rhov', 'Stfa2l1'],
                  'Dendritic cell': ['Cd209a', 'Irf5', 'Plbd1', 'Aif1', 'Cd209d'],
                  'Langerhans cell': ['Cd207', 'Mfge8', 'Cd74', 'Il1r2', 'Tnfaip2'],
                  'Mast cell': ['Cpa3', 'Cyp11a1', 'Cma1', 'Mcpt4', 'Tpsb2']
                  }

### WT1

In [ ]:
adata_WT1 = adata_all[adata_all.obs['batch'] == 'WT1']
sc.pp.filter_genes(adata_WT1, min_counts=1)

sc.pp.pca(adata_WT1, random_state=seed, n_comps=30)
sc.pp.neighbors(adata_WT1, random_state=seed, n_neighbors=int(0.5 * len(adata_WT1) ** 0.5), metric='cosine')
tk.tl.triku(adata_WT1, use_raw=False)

sc.tl.umap(adata_WT1, min_dist=0.3, random_state=seed)
sc.tl.leiden(adata_WT1, resolution=13, random_state=seed)

assign_cats(adata_WT1, dict_cats=dict_celltypes, min_score=0.4, quantile_gene_sel=0.7, key_added='cell_type')
adata_WT1 = adata_WT1[adata_WT1.obs['cell_type'] != 'Red blood cell']

sc.pp.subsample(adata_WT1, fraction=1, random_state=seed, copy=False)
sc.pl.umap(adata_WT1, color=['log1p_n_genes_by_counts', 'leiden',], alpha=0.5, ncols=3, legend_loc='on data')
sc.pl.umap(adata_WT1, color=['cell_type'], ncols=3)

In [ ]:
for key, val in dict_celltypes.items():
    print(key)
    sc.pl.umap(adata_WT1, color=['cell_type'] + [i for i in val if i in adata_WT1.var_names], legend_loc='on data', cmap=magma, use_raw=False, ncols=4) 

### WT2

In [ ]:
adata_WT2 = adata_all[adata_all.obs['batch'] == 'WT2']
sc.pp.filter_genes(adata_WT2, min_counts=1)

sc.pp.pca(adata_WT2, random_state=seed, n_comps=50)
sc.pp.neighbors(adata_WT2, random_state=seed, n_neighbors=int(0.5 * len(adata_WT2) ** 0.5), metric='cosine')
tk.tl.triku(adata_WT2, use_raw=False)

sc.tl.umap(adata_WT2, min_dist=0.3, random_state=seed)
sc.tl.leiden(adata_WT2, resolution=13, random_state=seed)

assign_cats(adata_WT2, dict_cats=dict_celltypes, min_score=0.45, quantile_gene_sel=0.75, key_added='cell_type')
adata_WT2 = adata_WT2[adata_WT2.obs['cell_type'] != 'Red blood cell']


sc.pp.subsample(adata_WT2, fraction=1, random_state=seed, copy=False)
sc.pl.umap(adata_WT2, color=['log1p_n_genes_by_counts', 'leiden'], alpha=0.5, ncols=2, legend_loc='on data')
sc.pl.umap(adata_WT2, color=['cell_type'], ncols=2, cmap=magma)

In [ ]:
for key, val in dict_celltypes.items():
    print(key)
    sc.pl.umap(adata_WT2, color=['cell_type'] + [i for i in val if i in adata_WT2.var_names], legend_loc='on data', cmap=magma, use_raw=False, ncols=4) 

### KO1

In [ ]:
adata_KOD11 = adata_all[adata_all.obs['batch'] == 'KO1']
sc.pp.filter_genes(adata_KOD11, min_counts=1)

sc.pp.pca(adata_KOD11, random_state=seed, n_comps=50)
sc.pp.neighbors(adata_KOD11, random_state=seed, n_neighbors=int(0.5 * len(adata_KOD11) ** 0.5), metric='cosine')
tk.tl.triku(adata_KOD11, use_raw=False)

sc.tl.umap(adata_KOD11, min_dist=0.3, random_state=seed)
sc.tl.leiden(adata_KOD11, resolution=11, random_state=seed)

assign_cats(adata_KOD11, dict_cats=dict_celltypes, min_score=0.4, quantile_gene_sel=0.7, key_added='cell_type')
adata_KOD11 = adata_KOD11[adata_KOD11.obs['cell_type'] != 'Red blood cell']

sc.pp.subsample(adata_KOD11, fraction=1, random_state=seed, copy=False)
sc.pl.umap(adata_KOD11, color=['log1p_n_genes_by_counts', 'leiden',], alpha=0.5, ncols=3, legend_loc='on data')
sc.pl.umap(adata_KOD11, color=['cell_type'], ncols=3)

In [ ]:
for key, val in dict_celltypes.items():
    print(key)
    sc.pl.umap(adata_KOD11, color=['cell_type'] + [i for i in val if i in adata_KOD11.var_names], legend_loc='on data', cmap=magma, use_raw=False, ncols=4) 

### KO2

In [ ]:
adata_KOD12 = adata_all[adata_all.obs['batch'] == 'KO2']
sc.pp.filter_genes(adata_KOD12, min_counts=1)

sc.pp.pca(adata_KOD12, random_state=seed, n_comps=35)
sc.pp.neighbors(adata_KOD12, random_state=seed, n_neighbors=int(0.5 * len(adata_KOD12) ** 0.5), metric='cosine')
tk.tl.triku(adata_KOD12, use_raw=False)

sc.tl.umap(adata_KOD12, min_dist=0.5, random_state=seed)
sc.tl.leiden(adata_KOD12, resolution=13, random_state=seed)

assign_cats(adata_KOD12, dict_cats=dict_celltypes, min_score=0.6, quantile_gene_sel=0.8, key_added='cell_type')
adata_KOD12 = adata_KOD12[adata_KOD12.obs['cell_type'] != 'Red blood cell']

sc.pp.subsample(adata_KOD12, fraction=1, random_state=seed, copy=False)
sc.pl.umap(adata_KOD12, color=['log1p_n_genes_by_counts', 'leiden',], alpha=0.5, ncols=3, legend_loc='on data')
sc.pl.umap(adata_KOD12, color=['cell_type'], ncols=3)

In [ ]:
for key, val in dict_celltypes.items():
    print(key)
    sc.pl.umap(adata_KOD12, color=['cell_type'] + [i for i in val if i in adata_KOD12.var_names], legend_loc='on data', cmap=magma, use_raw=False, ncols=4) 

## Comparison of clusters

This part shows which automatically-assigned populations appear within is samples. **Caution: some non-appearing populations may appear, but are too small**.

In [ ]:
isincelltype = pd.DataFrame('', index=sorted(dict_celltypes.keys()), columns=prefixes)

In [ ]:
for ct in isincelltype.index:
    for prefix in isincelltype.columns:
        if ct in set(eval(f'adata_{prefix}').obs['cell_type']):
            isincelltype.loc[ct, prefix] = 'x'

In [ ]:
isincelltype

### Adding cell types

In [ ]:
series = pd.Series(index = adata_all.obs_names)

series.loc[adata_WT1.obs_names] = adata_WT1.obs['cell_type']
series.loc[adata_WT2.obs_names] = adata_WT2.obs['cell_type']
series.loc[adata_KOD11.obs_names] = adata_KOD11.obs['cell_type']
series.loc[adata_KOD12.obs_names] = adata_KOD12.obs['cell_type']

adata_all.obs['cell_type'] = series

print(len(adata_all))
adata_all = adata_all[np.array([i !='nan' for i in adata_all.obs['cell_type'].values.astype(str)])].copy() # REMOVE RBC!!!!!!
print(len(adata_all))

sc.pp.filter_genes(adata_all, min_counts=1)

## WT dataset analysis

We are going to join the populations using `harmony`. We also tried `bbknn` and `scnaorama`, but `harmony` yielded the best results.

In [ ]:
adata_WT_harmony = adata_all[adata_all.obs['batch'].isin(['WT1', 'WT2'])]
sc.pp.filter_genes(adata_WT_harmony, min_counts=1)

In [ ]:
sc.pp.pca(adata_WT_harmony, random_state=seed, n_comps=50)
sce.pp.harmony_integrate(adata_WT_harmony, key='batch', max_iter_harmony=50, plot_convergence=True)
sc.pp.neighbors(adata_WT_harmony, random_state=seed, n_neighbors=int(len(adata_WT_harmony) ** 0.5), metric='cosine', use_rep='X_pca_harmony')
tk.tl.triku(adata_WT_harmony, use_raw=False)

In [ ]:
sc.tl.umap(adata_WT_harmony, min_dist=0.5, random_state=seed)
sc.tl.leiden(adata_WT_harmony, resolution=0.5, random_state=seed)

In [ ]:
sc.pp.subsample(adata_WT_harmony, fraction=1, random_state=seed, copy=False)
sc.pl.umap(adata_WT_harmony, color=['batch', 'log1p_n_genes_by_counts', 'leiden', 'cell_type'], ncols=2, alpha=0.5)

In [ ]:
sc.pl.umap(adata_WT_harmony, color=['cell_type'], ncols=2, legend_loc='on data')

We see that there is a fair integration of WT1 and WT2. WT1 seems to have more RBC and Neutrophils compared to WT2, but nothing really special.

## KOD dataset analysis

In [ ]:
adata_KOD_harmony = adata_all[adata_all.obs['batch'].isin(['KO1', 'KO2'])]
sc.pp.filter_genes(adata_KOD_harmony, min_counts=1)

In [ ]:
sc.pp.pca(adata_KOD_harmony, random_state=seed, n_comps=50)
sce.pp.harmony_integrate(adata_KOD_harmony, key='batch', max_iter_harmony=50, plot_convergence=True)
sc.pp.neighbors(adata_KOD_harmony, random_state=seed, n_neighbors=int(0.5 * len(adata_KOD_harmony) ** 0.5), metric='cosine', use_rep='X_pca_harmony')
tk.tl.triku(adata_KOD_harmony, use_raw=False)

In [ ]:
sc.tl.umap(adata_KOD_harmony, min_dist=0.4, random_state=seed)
sc.tl.leiden(adata_KOD_harmony, resolution=1.3, random_state=seed)

In [ ]:
sc.pl.umap(adata_KOD_harmony, color=['batch', 'log1p_n_genes_by_counts', 'leiden', 'cell_type'], ncols=2, alpha=0.5)

In [ ]:
sc.pl.umap(adata_KOD_harmony, color=['cell_type'], ncols=2, legend_loc='on data')

KOD11 seems to integrate worse with KOD12 than WT1 with WT2. KOD12 seems to have more RBCs, whereas KOD11 shows a greater numer of fibroblasts, and fewer keratinocyte populations.

## Whole dataset analysis

In [ ]:
adata_all_harmony = adata_all.copy()
sc.pp.pca(adata_all_harmony, random_state=seed, n_comps=30)
sce.pp.harmony_integrate(adata_all_harmony, key='batch', max_iter_harmony=50, plot_convergence=True, sigma=0.05)
sc.pp.neighbors(adata_all_harmony, random_state=seed, n_neighbors=int(0.2 * len(adata_all_harmony) ** 0.5), metric='cosine', use_rep='X_pca_harmony')
tk.tl.triku(adata_all_harmony, use_raw=False)

sc.tl.umap(adata_all_harmony, min_dist=0.7, random_state=seed)

sc.tl.leiden(adata_all_harmony, resolution=4, random_state=seed)
assign_cats(adata_all_harmony, dict_cats=dict_celltypes, min_score=0.6, quantile_gene_sel=0.8, key_added='cell_type_whole')

# sc.pp.subsample(adata_all_harmony, fraction=1, random_state=seed, copy=False)
# sc.pl.umap(adata_all_harmony, color=['batch', 'status', 'log1p_n_genes_by_counts', 'leiden', 'cell_type', 'cell_type_whole'], alpha=0.5, ncols=2)
# sc.pl.umap(adata_all_harmony, color=[f'is_{prefix}' for prefix in prefixes], alpha=0.5, ncols=2)

In [ ]:
sc.pl.umap(adata_all_harmony, color=['cell_type', 'cell_type_whole', 'leiden'], alpha=0.7, ncols=1)

### Analysis of positive results

In [ ]:
# IL1RL1 population
# The protein encoded by this gene is a member of the interleukin 1 receptor family. 
# Studies of the similar gene in mouse suggested that this receptor can be induced by proinflammatory stimuli, 
# and may be involved in the function of helper T cells.

sc.pl.umap(adata_all_harmony, color=['batch', 'cell_type', 'Il1rl1'], alpha=0.5, ncols=2, cmap=magma)
sc.pl.umap(adata_all_harmony, color=[f'is_{prefix}' for prefix in prefixes], alpha=0.5, ncols=4)

sc.pl.umap(adata_WT1, color=['Il1rl1', 'cell_type'], cmap=magma, use_raw=False, ncols=4) 
sc.pl.umap(adata_WT2, color=['Il1rl1', 'cell_type'], cmap=magma, use_raw=False, ncols=4) 
sc.pl.umap(adata_KOD11, color=['Il1rl1', 'cell_type'], cmap=magma, use_raw=False, ncols=4) 
sc.pl.umap(adata_KOD12, color=['Il1rl1', 'cell_type'], cmap=magma, use_raw=False, ncols=4) 

In [ ]:
# Neutrophil and * population

sc.pl.umap(adata_all_harmony, color=['batch', 'cell_type', 'S100a9', 'Acod1', 'Il1f9', 'Rhov'], alpha=0.5, ncols=2, cmap=magma)
sc.pl.umap(adata_all_harmony, color=[f'is_{prefix}' for prefix in prefixes], alpha=0.5, ncols=4)

list_per = []
for adata in prefixes:
    a = len(eval(f'adata_{adata}')[eval(f'adata_{adata}').obs['cell_type'].isin(['Neutrophil', 'Neutrophil*'])])
    b = len(eval(f'adata_{adata}'))
    list_per.append(100 * a / b)

df = pd.DataFrame({'x': prefixes, 'y': list_per})

fig,ax = plt.subplots(1, 1, figsize=(4, 1))
sns.barplot(x='x', y='y', data=df, ax=ax)

In [ ]:
# NK cells

sc.pl.umap(adata_all_harmony, color=['batch', 'cell_type', 'Cd3d', 'Nkg7', 'Trdv4'], alpha=0.5, ncols=2, cmap=magma)
sc.pl.umap(adata_all_harmony, color=[f'is_{prefix}' for prefix in prefixes], alpha=0.5, ncols=4)

list_per = []
for adata in prefixes:
    a = len(eval(f'adata_{adata}')[eval(f'adata_{adata}').obs['cell_type'].isin(['NK cell', 'T cell', 'T cell (ILC/gd)?'])])
    b = len(eval(f'adata_{adata}'))
    list_per.append(100 * a / b)

df = pd.DataFrame({'x': prefixes, 'y': list_per})

fig,ax = plt.subplots(1, 1, figsize=(4, 1))
sns.barplot(x='x', y='y', data=df, ax=ax)

## Analysis of Macrophage populations

In [ ]:
adata_all_harmony_mono_macro = adata_all_harmony[adata_all_harmony.obs['cell_type'].isin(['Monocyte', 'Macrophage'])]

sc.pp.filter_genes(adata_all_harmony_mono_macro, min_cells=5)
sc.pp.pca(adata_all_harmony_mono_macro, random_state=seed, n_comps=30)
sce.pp.harmony_integrate(adata_all_harmony_mono_macro, key='batch', max_iter_harmony=50, plot_convergence=True, sigma=0.05)
sc.pp.neighbors(adata_all_harmony_mono_macro, random_state=seed, n_neighbors=int(len(adata_all_harmony_mono_macro) ** 0.5), metric='cosine', use_rep='X_pca_harmony')
tk.tl.triku(adata_all_harmony_mono_macro, use_raw=False)

sc.tl.umap(adata_all_harmony_mono_macro, min_dist=0.3, random_state=seed)
sc.tl.leiden(adata_all_harmony_mono_macro, resolution=1.3, random_state=seed)

sc.pp.subsample(adata_all_harmony_mono_macro, fraction=1, random_state=seed, copy=False)
sc.pl.umap(adata_all_harmony_mono_macro, color=['batch', 'status', 'log1p_n_genes_by_counts', 'leiden', 'cell_type'], alpha=0.5, ncols=2)
sc.pl.umap(adata_all_harmony_mono_macro, color=[f'is_{prefix}' for prefix in prefixes], alpha=0.5, ncols=2)

# Adata saving

In [ ]:
adata_all_harmony.write_h5ad('adatas/adata_all_harmony.h5')